In [2]:
import pandas as pd
import numpy as np

import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensim_models
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from text_analysis_module import pr1
from text_analysis_module import pr2
from text_analysis_module import create_text_matrix
from hashtag_analysis_module import create_hashtag_matrix
from interacting_user_analysis_module import create_interacting_users_matrix


USERNAME = 'pleasedshibe1' #TODO: Get from front end
header = ['username', 'name', 'description', 'number of tweets', 'followers', 'verified', 'tweets', 'links', 'hashtags', 'interacting users']
df1 = pd.read_csv(f"../{USERNAME}.csv", header=None, names=header, index_col=False) #TODO: Get column directly from query
df1


def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i


df1 = df1[df1['tweets'].notna()]
df1.reset_index(drop=True, inplace=True)


In [3]:
df1

,username,name,description,number of tweets,followers,verified,tweets,links,hashtags,interacting users
0,stvncrdbl,BS+ Stvn,Rocket league player for @BScompetition Busin...,6148,655,False,The issue I have with improving or being notic...,[['https://clips.twitch.tv/AntediluvianFancyHe...,[],"['paralakz', 'paralakz', 'jakefawcett3', 'jake..."
1,Redlion_56,redlionlol #advyout,proud owner of $36 | creo stan and minecraft e...,12940,221,False,@ProCyGFX @HariKama_ @zetexkindasucks @MindCap...,[['https://twitter.com/Comgamingnz/status/1473...,[],"['ProCyGFX', 'ProCyGFX', 'HariKama_', 'zetexki..."
2,ItzDaMatrix,/\/\atrix,Rocket League co-lead for @TheAsylumClique | S...,586,133,False,I dunno why but I guess this name better than ...,[['https://twitter.com/MaximusAemulus/status/1...,[['turbolan']],"['miladmirg', 'miladmirg', 'MaximusAemulus', '..."
3,17BluJay17,Jay,NaN,1,0,False,@ShquareSMP :0 shibe that is rude,[],[],"['ShquareSMP', 'ShquareSMP']"
4,regal_pasta,Regal Pasta,NaN,3,5,False,@blazeshardcat tabaxi + triton = catfish @kath...,[],[],"['blazeshardcat', 'blazeshardcat', 'katherinej..."
...,...,...,...,...,...,...,...,...,...,...
85,Eizochupapi,Eizo_97,I be on twitter,136,8,False,@Hannahiku @ShquareSMP Hannahiku is cool dont ...,"[['https://discord.gg/VB4stagy'], ['https://di...","[['MyTwitterAnniversary'], ['TubboforPresident...","['Hannahiku', 'Hannahiku', 'ShquareSMP', 'Shqu..."
86,iisome_loser,Zoring•,.-.,1,0,False,HI BITCHES,[],[],[]
87,TheCreatorsSMP,The Creators SMP,Hello We are The Creators SMP we are an smp fo...,33,188,False,@ProbablyKaz HAPPY BIRTHDAY FROM CSMP @Probabl...,[['https://twitter.com/MayBearYT/status/138978...,[['TikTok']],"['ProbablyKaz', 'ProbablyKaz', 'ProbablyKaz', ..."
88,Sorbelato_,straw,Comms Open! • 19 • They/Them •🇲🇽•currently: 💤,926,164,False,@Elixer_mv It was so much fun though! A little...,"[['https://www.twitch.tv/sorbelato'], ['https:...",[],"['Elixer_mv', 'Elixer_mv', 'Debunkadunk', 'Deb..."


In [45]:
#1 Text similarity
df1 = df1[df1['tweets'].notna()]
# df1 = df1[df1['interacting users'].notna()]
df1.reset_index(drop=True, inplace=True)
df1


,username,name,description,number of tweets,followers,verified,tweets,links,hashtags,interacting users
0,ItzDaMatrix,/\/\atrix,Rocket League co-lead for @TheAsylumClique | S...,583,130,False,@TOGfoxes @AOC_Gaming @tnigh_ @yayazuk @NZXT 💜...,"[['https://youtu.be/j5a0jTc9S10'], ['https://t...","[['turbolan'], ['CoinBeastSweepstakes']]","['TOGfoxes', 'TOGfoxes', 'AOC_Gaming', 'tnigh_..."
1,Gogelss,Gogels :0,Small streamer twitch.tv/googels meow he/him,23,4,False,cleaning it my birthday is next day :DDD They ...,"[['https://www.twitch.tv/googels', 'https://tw...",[],"['Gogelss', 'Gogelss', 'ShquareSMP', 'ShquareS..."
2,Nuko79113670,Nuko,Its me Nuko,1,12,False,Rawr,[],[],[]
3,Entropy_SMP,Entropy,en·tro·py - lack of order or predictability; g...,18,117,False,@MackCommentator BET @MackCommentator I'm this...,[['https://twitter.com/Entropy_SMP/status/1471...,"[['EntropySMP', 'animation', 'SMP'], ['Entropy...","['MackCommentator', 'MackCommentator', 'MackCo..."
4,biiguhh,𝕓𝕣𝕦𝕙,kazuha main - ar 52 - eu,783,32,False,#fortniteflipped @uhhHadZ @stupidcoupe @uhhHad...,[['https://twitter.com/biiguhh/status/14417339...,"[['fortniteflipped'], ['PS5Share', 'GenshinImp...","['uhhHadZ', 'uhhHadZ', 'uhhHadZ', 'uhhHadZ', '..."
...,...,...,...,...,...,...,...,...,...,...
159,Neon34_,Neon,Just a Narwhal that requires CONTENT | Banner:...,3093,86,False,SPIDER-MAN TIME BABYYYY VAMOS KRU I AM A KRU F...,[['https://www.youtube.com/watch?v=kABiIlGhKCE...,"[['CancelDebunky'], ['VCT'], ['AnimalCrossing'...","['ALemonmc', 'ALemonmc', 'Neon34_', 'Neon34_',..."
160,NotAshAttacks,Santa Ash 🎁,organizer of @OutskirtsSMP & @CreatorCores | 1...,1816,251,False,Mans playing Minecraft PE.. the memories 😇 @Sn...,[['https://twitter.com/timeclova/status/147206...,"[['cancelcancelculture'], ['SLCSHForSMPOnAir']...","['Snaplu_', 'Snaplu_', 'TheDerpyKatLIVE', 'Twi..."
161,YupItzPhrogy,Phrogy,who can game??? 😎 not me... 😐🤭 • they/th...,67,6,False,"I really want the glare added to Minecraft, it...","[['https://www.twitch.tv/YupItzPhrogy?sr=a'], ...","[['persona', 'drawing', 'art', 'nonbinary', 'a...","['discord', 'discord', 'meowsiris', 'PEBBLEBRA..."
162,tregingtons,Treg,I like content creating sometimes a VERY hand...,3004,209,False,I desperately need someone sarcastic and menac...,[['https://twitter.com/jonahwill/status/147297...,"[['EarthboundSMP'], ['CopperGolemSupremacy'], ...","['Vdubble1', 'Vdubble1', 'TehPitchMaker', 'tre..."


In [46]:

text_matrix = create_text_matrix(df1)


In [47]:
hashtag_matrix = create_hashtag_matrix(df1)

/home/liam_mckenna/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [48]:
interacting_users_matrix = create_interacting_users_matrix(df1)

In [49]:
interacting_users_matrix

[[0,
  0.9090199827963208,
  0.9814933774925495,
  0.9040861687657289,
  0.9687613282070161,
  0.946658236684857,
  0.8986611150713242,
  0.9705490245597693,
  0.9620623806374335,
  0.9380986721448893,
  0.9700098828197976,
  0.8805402653901614,
  0.9578623457619176,
  0.9132918186347099,
  0.855755152932691,
  0.8578736424677085,
  0.9158069074340229,
  0.9814933774925495,
  0.9476725013834149,
  0.9439021520477899,
  0.9624261936008636,
  0.9570234784834639,
  0.8769859762324052,
  0.9285013013616178,
  0.8850362141499711,
  0.9263327847097108,
  0.8894017765847168,
  0.9508665947544162,
  0.8895224950744833,
  0.9608963517646628,
  0.9708347040857461,
  0.9654549853006458,
  0.9708572246352243,
  0.9461784260789562,
  0.9177951421964821,
  0.9153516012251012,
  0.9034478507995217,
  0.9709482695808713,
  0.9686526300599186,
  0.9706142747661333,
  0.9692262977057884,
  0.960694384490199,
  0.9705372989561116,
  0.9661171559805991,
  0.9661499336234324,
  0.9524096451489157,
  0.9547

In [51]:
print(len(interacting_users_matrix), 'by', len(interacting_users_matrix[0]))
print(len(hashtag_matrix), 'by', len(hashtag_matrix[0]))
print(len(text_matrix), 'by', len(text_matrix[0]))
text_mtx = np.matrix(text_matrix)
hashtag_mtx = np.matrix(hashtag_matrix)
interact_mtx = np.matrix(interacting_users_matrix)

similarity_matrix = (.4 * text_mtx) + (.1 * hashtag_mtx) + (.5 * interact_mtx)


164 by 164
164 by 164
164 by 164


In [52]:
interact_mtx

matrix([[0.        , 0.90901998, 0.98149338, ..., 0.90555879, 0.96862972,
         0.91194749],
        [0.90901998, 0.        , 0.8327516 , ..., 0.75089905, 0.94055619,
         0.80835129],
        [0.98149338, 0.8327516 , 0.        , ..., 0.94226497, 0.98790829,
         0.96101594],
        ...,
        [0.90555879, 0.75089905, 0.94226497, ..., 0.        , 0.93829474,
         0.81287652],
        [0.96862972, 0.94055619, 0.98790829, ..., 0.93829474, 0.        ,
         0.94246893],
        [0.91194749, 0.80835129, 0.96101594, ..., 0.81287652, 0.94246893,
         0.        ]])

In [53]:
similarity_matrix

matrix([[ 1.16415377e-11,  8.26231140e-01,  8.37755179e-01, ...,
          8.25556427e-01,  8.58267941e-01,  8.29733775e-01],
        [ 8.26231140e-01,  1.00000000e-01,  7.00136072e-01, ...,
          6.72955668e-01,  6.64815828e-01,  5.70535415e-01],
        [ 8.37755179e-01,  7.00136072e-01,  1.00000000e-01, ...,
          7.38017433e-01,  7.81507454e-01,  7.60806227e-01],
        ...,
        [ 8.25556450e-01,  6.72955668e-01,  7.38017433e-01, ...,
         -8.32369373e-10,  7.84115311e-01,  6.06824902e-01],
        [ 8.58267941e-01,  6.64815810e-01,  7.81507418e-01, ...,
          7.84115287e-01,  0.00000000e+00,  7.09372404e-01],
        [ 8.29733775e-01,  5.70534771e-01,  7.60806251e-01, ...,
          6.06809799e-01,  7.09372404e-01,  1.00000000e-01]])

In [54]:
#TODO: seperate into files